In [3]:
%matplotlib inline

import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn_pandas import DataFrameMapper
import warnings

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

In [4]:
# data_no_unknown.loc[np.logical_not(data_no_unknown.pedestrian_location == data_no_unknown.pedestrian_location), 'pedestrian_location'] = ''
# data_no_unknown.loc[np.logical_not(data_no_unknown.pedestrian_movement == data_no_unknown.pedestrian_movement), 'pedestrian_movement'] = ''


In [5]:
casualty_data = pandas.read_csv('./casualty_train.csv')
data_no_unknown = casualty_data.loc[casualty_data.age != 'Unknown']
data_no_unknown.loc[np.logical_not(data_no_unknown.pedestrian_location == data_no_unknown.pedestrian_location), 'pedestrian_location'] = ''
data_no_unknown.loc[np.logical_not(data_no_unknown.pedestrian_movement == data_no_unknown.pedestrian_movement), 'pedestrian_movement'] = ''
data_no_unknown.severe = data_no_unknown.severe.astype('int')
data_no_unknown.head()

/Users/guydavidson/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/guydavidson/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,casualty_class,gender,age,severe,pedestrian_location,pedestrian_movement,travel,year
0,passenger,female,33,0,,,motorbike,2007
1,passenger,female,20,0,,,car,2005
2,passenger,male,52,0,,,car,2006
3,passenger,female,17,0,,,pedestrian,2012
4,passenger,female,20,0,,,motorbike,2010


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_no_unknown, test_size=0.1, random_state=RANDOM_SEED,
                               stratify=data_no_unknown.casualty_class, ) #  train_size = 0.25,
train.shape, test.shape

((194949, 8), (21661, 8))

In [7]:
# casualty_data.loc[casualty_data.age == 'Unknown'].groupby('casualty_class').sum()

In [8]:
data_mapper = DataFrameMapper([
    ('casualty_class', sklearn.preprocessing.LabelBinarizer()),
    ('gender', sklearn.preprocessing.LabelBinarizer()),
    (['age'], sklearn.preprocessing.StandardScaler()),
#     ('pedestrian_location', sklearn.preprocessing.LabelBinarizer()),
#     ('pedestrian_movement', sklearn.preprocessing.LabelBinarizer()),
    ('travel', sklearn.preprocessing.LabelBinarizer()),
    (['year'], sklearn.preprocessing.StandardScaler()),
])

transformed_data = data_mapper.fit_transform(train.copy())
print(data_mapper.transformed_names_)
transformed_data[:5]

/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


['casualty_class_driver', 'casualty_class_passenger', 'casualty_class_pedestrian', 'gender', 'age', 'travel_bicycle', 'travel_bus', 'travel_car', 'travel_motorbike', 'travel_other', 'travel_pedestrian', 'travel_taxi', 'year']


array([[ 1.        ,  0.        ,  0.        ,  1.        , -1.099384  ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.19920703],
       [ 0.        ,  1.        ,  0.        ,  0.        , -0.31618757,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.85600581],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.22602534,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.51280458],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.88873001,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.85600581],
       [ 1.        ,  0.        ,  0.        ,  1.        ,  1.73217232,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.86000031]])

In [15]:
from sklearn.model_selection import cross_val_score

np.random.seed(33)

# Fun fact: if you fit a logit without class weights, it always predicts 0. Womp womp.

pipe = sklearn.pipeline.Pipeline([
    ('featurize', data_mapper),
#     ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression)),
    ('svm', sklearn.svm.SVC(cache_size=1024, kernel='rbf', class_weight='balanced', verbose=True))
#     ('logit', sklearn.linear_model.LogisticRegression(class_weight='balanced', verbose=True))
])

# pipe.fit(train, train.severe)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
        cv_scores = np.round(cross_val_score(pipe, X=train, y=train.severe, scoring='f1', cv=3), 3)
        print(np.average(cv_scores), cv_scores)
    
#     for k in range(6, 16):
#         pipe.set_params(select__k=k)
#         shuffled = train.sample(frac=1)
#         cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.severe, scoring='f1', cv=5), 3)
#         print(k, np.average(cv_scores), cv_scores)

[LibSVM][LibSVM][LibSVM]0.262333333333 [ 0.261  0.264  0.262]


In [29]:
from sklearn.model_selection import cross_val_score

np.random.seed(33)

# Fun fact: if you fit a logit without class weights, it always predicts 0. Womp womp.

logit_pipe = sklearn.pipeline.Pipeline([
    ('featurize', data_mapper),
#     ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression)),
#     ('svm', sklearn.svm.SVC(cache_size=1024, kernel='linear', class_weight='balanced', verbose=True))
    ('logit', sklearn.linear_model.LogisticRegression(verbose=True))
])

# pipe.fit(train, train.severe)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
#     weights = train.severe + 1
#     print(train.shape, weights.shape)
#     cv_scores = np.round(cross_val_score(logit_pipe, X=train, y=train.severe, scoring='f1', cv=3, 
# #                                          fit_params=dict(logit__sample_weight=weights)
#                                         ), 3)
#     print(np.average(cv_scores), cv_scores)
    
    for one_class_weight in np.random.rand:
        pipe.set_params(select__k=k)
        shuffled = train.sample(frac=1)
        cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.severe, scoring='f1', cv=5), 3)
        print(k, np.average(cv_scores), cv_scores)

(194949, 8) (194949,)
[LibLinear][LibLinear][LibLinear]0.269333333333 [ 0.268  0.271  0.269]


## Trying to randomly search over 1-class weights

In [31]:
from sklearn.metrics import classification_report

logit_pipe.fit(train, train.severe)
print(classification_report(logit_pipe.predict(test), test.severe,))


/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[LibLinear]             precision    recall  f1-score   support

          0       0.60      0.93      0.73     12406
          1       0.64      0.16      0.26      9255

avg / total       0.62      0.60      0.53     21661



In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

np.random.seed(RANDOM_SEED)

logit_search_pipe = sklearn.pipeline.Pipeline([
    ('featurize', data_mapper),
#     ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression)),
    ('logit', sklearn.linear_model.LogisticRegression(verbose=True))
])

from collections import namedtuple
ClassWeightSampler = namedtuple('ClassWeightSampler', 'rvs')

def sample_class_weights(loc=0, scale=1, size=1, random_state=None):
    return {0: 1, 1: uniform.rvs(loc=1, scale=10, size=size, random_state=random_state)}

sampler = ClassWeightSampler(rvs=sample_class_weights)

# report function from http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    search_cv = RandomizedSearchCV(logit_search_pipe, n_iter=10, scoring='f1', n_jobs=4, 
                                   verbose=1, random_state=RANDOM_SEED,
                                   param_distributions={'logit__class_weight': sampler,
                                                        'logit__C': uniform(loc=1, scale=10) 
                                                       })
    search_cv.fit(X=train, y=train.severe)
    report(search_cv.cv_results_)

#     for one_class_weight in np.random.rand:
#         pipe.set_params(select__k=k)
#         shuffled = train.sample(frac=1)
#         cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.severe, scoring='f1', cv=5), 3)
#         print(k, np.average(cv_scores), cv_scores)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   45.5s finished


[LibLinear]Model with rank: 1
Mean validation score: 0.274 (std: 0.002)
Parameters: {'logit__C': 9.8062818421524138, 'logit__class_weight': {0: 1, 1: array([ 5.94063468])}}

Model with rank: 2
Mean validation score: 0.274 (std: 0.002)
Parameters: {'logit__C': 7.8045080473103923, 'logit__class_weight': {0: 1, 1: array([ 5.86588127])}}

Model with rank: 3
Mean validation score: 0.272 (std: 0.002)
Parameters: {'logit__C': 3.4851012743772993, 'logit__class_weight': {0: 1, 1: array([ 5.49975421])}}



In [12]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, mutual_info_score

test_predictions = search_cv.best_estimator_.predict(test)

for metric in (classification_report, accuracy_score, roc_auc_score, mutual_info_score):
    print(metric.__name__)
    print(metric(test_predictions, test.severe))


classification_report
             precision    recall  f1-score   support

          0       0.74      0.92      0.82     15597
          1       0.47      0.19      0.27      6064

avg / total       0.67      0.71      0.67     21661

accuracy_score
0.714509948756
roc_auc_score
0.552748239885
mutual_info_score
0.0105637229065


/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [14]:
new_test_data = pandas.read_csv('./casualty_test.csv')
new_test_data = new_test_data.loc[new_test_data.age != 'Unknown']
new_test_data.loc[np.logical_not(new_test_data.pedestrian_location == new_test_data.pedestrian_location), 'pedestrian_location'] = ''
new_test_data.loc[np.logical_not(new_test_data.pedestrian_movement == new_test_data.pedestrian_movement), 'pedestrian_movement'] = ''
new_test_data.severe = new_test_data.severe.astype('int')
new_test_data.head()

,casualty_class,gender,age,severe,pedestrian_location,pedestrian_movement,travel,year
0,passenger,male,32,1,,,pedestrian,2013
1,passenger,male,34,0,,,pedestrian,2008
2,passenger,female,57,0,,,car,2010
3,passenger,male,37,0,,,car,2011
4,passenger,female,23,0,,,car,2007


In [16]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, mutual_info_score

new_test_predictions = search_cv.best_estimator_.predict(new_test_data)

for metric in (classification_report, accuracy_score, roc_auc_score, mutual_info_score):
    print(metric.__name__)
    print(metric(new_test_predictions, new_test_data.severe))


classification_report
             precision    recall  f1-score   support

          0       0.75      0.92      0.82     39071
          1       0.49      0.19      0.27     15206

avg / total       0.67      0.72      0.67     54277

accuracy_score
0.717449380032
roc_auc_score
0.556360603485
mutual_info_score
0.0120545312642


/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

np.random.seed(RANDOM_SEED)

ridge_search_pipe = sklearn.pipeline.Pipeline([
    ('featurize', data_mapper),
    ('ridge', sklearn.linear_model.RidgeClassifierCV())
])

from collections import namedtuple
ClassWeightSampler = namedtuple('ClassWeightSampler', 'rvs')

def sample_class_weights(loc=0, scale=1, size=1, random_state=None):
    return {0: 1, 1: uniform.rvs(loc=1, scale=20, size=size, random_state=random_state)}

sampler = ClassWeightSampler(rvs=sample_class_weights)

# report function from http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    search_cv = RandomizedSearchCV(logit_search_pipe, n_iter=10, scoring='recall', n_jobs=4, 
                                   verbose=1, random_state=RANDOM_SEED,
                                   param_distributions={'ridge__class_weight': sampler,
#                                                         'logit__C': uniform(loc=1, scale=10) 
                                                       })
    search_cv.fit(X=train, y=train.severe)
    report(search_cv.cv_results_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/guydavidson/anaconda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/guydavidson/anaco